# LLM model

In [ ]:
!pip install openai==0.28.1

In [7]:
import requests

url = "http://127.0.0.1:1234/v1/chat/completions"
payload = {
    "model": "Qwen3-4B",  # ใส่ชื่อที่ถูกต้องที่ LM Studio ขึ้นไว้
    "messages": [
        {"role": "user", "content": "แนะนำตัวเองหน่อย"}
    ]
}
headers = {"Authorization": "Bearer lm-studio"}
response = requests.post(url, json=payload, headers=headers)
print(response.json())

{'id': 'chatcmpl-u5c6t1mmskfenni7fuari', 'object': 'chat.completion', 'created': 1749987638, 'model': 'qwen3-4b', 'choices': [{'index': 0, 'logprobs': None, 'finish_reason': 'stop', 'message': {'role': 'assistant', 'content': "<think>\nOkay, the user asked me to introduce myself. I need to start by stating my name clearly. I should mention that I'm Qwen, developed by Alibaba Cloud. It's important to highlight my capabilities in answering questions, creating content, and handling various tasks.\n\nI should explain that I can communicate in multiple languages, which is a key feature. Also, I need to emphasize my ability to understand and generate text based on the user's input. Maybe include examples of what I can do, like writing stories, explaining concepts, or helping with problem-solving.\n\nIt's good to mention that I'm designed to be helpful and efficient, so users can rely on me for assistance. I should also invite them to ask questions or share what they need help with. Keep the 

In [ ]:
import openai
print(openai.__version__)
import re

# กำหนดค่า endpoint สำหรับ LM Studio API (หรือ Ollama) (localhost)
openai.api_base = "http://127.0.0.1:1234/v1"
openai.api_key = "lm-studio"

# สร้าง system message เพื่อบอก persona ของบอททุกครั้ง
system_message = {"role": "system", "content": "คุณเป็นแชทบอทผู้ช่วยที่พูดสุภาพและกระชับ"}

# ฟังก์ชันสำหรับลบ <think>...</think> ออกจากคำตอบ (เพื่อ clean เฉพาะเนื้อหาที่ผู้ใช้ต้องการ)
def clean_assistant_reply(text):
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()

memory = []  # เก็บเฉพาะข้อความ user/assistant ตามจำนวน memory ล่าสุด
log = []     # เก็บ log ถาม-ตอบทั้งหมด

# ฟังก์ชันสร้าง messages สำหรับแต่ละรอบสนทนา (จำ N รอบหลังสุด)
def build_messages(memory, user_input, max_memory=5):
    relevant = memory[-(max_memory*2):]    # ดึง 5 คู่ user+assistant ล่าสุด (รวม 10 รายการ)
    messages = [system_message]            # ใส่ system message ที่จุดเริ่มต้น
    for i, msg in enumerate(relevant):
        role = "user" if i % 2 == 0 else "assistant"
        messages.append({"role": role, "content": msg})
    messages.append({"role": "user", "content": user_input})   # เพิ่มข้อความล่าสุดจาก user
    return messages

print("---- พิมพ์ 'exit' เพื่อออก ----")
while True:
    user_input = input("คุณ: ")
    if user_input.lower() == "exit":
        break

    # สร้าง messages สำหรับส่งไปที่ API
    messages = build_messages(memory, user_input, max_memory=5)

    # ส่งข้อความไปที่ LLM ผ่าน OpenAI compatible API ของ LM Studio
    response = openai.ChatCompletion.create(
        model="qwen3-4b",  # ใส่ชื่อ model ที่ตรงกับ LM Studio
        messages=messages,
        temperature=0.7,
    )
    # ดึงคำตอบออกมาและลบ <think>...</think>
    reply = response['choices'][0]['message']['content']
    reply_clean = clean_assistant_reply(reply)
    print("AI:", reply_clean)

    # อัปเดต memory (แค่ 5 คู่หลัง) และ log (เก็บทั้งหมด)
    memory.append(user_input)
    memory.append(reply_clean)
    log.append(("user", user_input))
    log.append(("assistant", reply_clean))

# แสดง log ถาม-ตอบทั้งหมดหลัง exit
print("\n===== Conversation Log =====")
for i, (role, msg) in enumerate(log, 1):
    who = "คุณ" if role == "user" else "AI"
    print(f"{i:02d}. {who}: {msg}")

# แสดง memory 5 คู่ล่าสุด (10 ข้อความสุดท้าย)
print("\n===== Memory (5 pairs ล่าสุด) =====")
for i in range(len(memory)-10, len(memory), 2):
    if i >= 0:
        print(f"Q: {memory[i]}")
        print(f"A: {memory[i+1]}")


---- พิมพ์ 'exit' เพื่อออก ----
AI: การจับคู่โค้ก (Coca-Cola) กับอาหารหรือเครื่องดื่มต่างๆ สามารถทำให้รสชาติชัดเจนและอร่อยได้มากขึ้น ลองพิจารณาแนวคิดเหล่านี้:

### **1. อาหารที่มีความเค็มหรือสัมผัสได้ถึงความเปรี้ยว**  
- **แครอทต์ชิ้ง (Carrotsticks)**: เนื้อนุ่มและรสชาติกลางๆ ช่วยให้รสหวานของโค้กเด่นขึ้น  
- **ขนมปังฟาร์เมอร์ (Farmer's Bread)**: ความเค็มจากเนยและเกลือช่วยสมดุลรสหวาน  
- **พิซซาสีดำ (Black Pizza)**: ถั่วและเครื่องเทศที่มีความเผ็ดเล็กน้อย ให้ความรู้สึกใหม่ๆ  

### **2. อาหารที่มีความขมหรือเปรี้ยว**  
- **เกลือแร่ (Sea Salt)**: ใช้กับขนมปังหรือแครอทชิ้งเพื่อเพิ่มรสเค็มและกล้ามเนื้อของโค้ก  
- **นมข้น (Cinnamon Rolls)**: ความหวานจากน้ำตาลและพริกไทยช่วยให้รสชาติสมดุล  

### **3. อาหารที่มีความเผ็ดหรือกระชับ**  
- **ไส้กรอกผัด (Stir-fried Meat)**: เผ็ดเล็กน้อยช่วยให้รสหวานของโค้กไม่เกินหัวใจ  
- **ข้าวผัด (Stir-fried Rice)**: ความเผ็ดจากเครื่องเทศและน้ำมันพืชทำให้รสชาติสดใส  

### **4. เครื่องดื่มที่เสริมรสชาติ**  
- **นมถั่ว (Almond Milk)**: ความขมของนมถั่วช่วยให้รสหวานของโ